# Paso 1: Preprocesamiento del Dataset

El primer paso en nuestro proceso es el preprocesamiento del dataset. Este dataset se encuentra en formato CSV y requiere de varias transformaciones antes de poder ser utilizado para entrenar nuestro modelo.

1. Eliminación de columnas no útiles, valores nulos y duplicados: Algunas columnas como 'Unnamed: 0' y 'Flow ID' no aportan información relevante para el entrenamiento del modelo, por lo que serán eliminadas. Los valores nulos (o NaN) y los elementos duplicados pueden afectar la precisión de nuestro modelo, por lo que también serán eliminados.

In [1]:
import pandas as pd

#Cargar el dataset, eliminar columnas irrelevantes, valores nulos y elementos repetidos
df = pd.read_csv('DataSets//CICDDoS2019//01-12//Syn(1).csv')
df = df.drop(['Unnamed: 0', 'Flow ID'], axis=1)
df = df.dropna()
df = df.drop_duplicates()

2. Codificación de datos categóricos: Tenemos varias columnas (como la IP) que contienen datos categóricos que deben ser codificados, por ejemplo, numéricamente.

In [2]:
import ipaddress
from sklearn.preprocessing import LabelEncoder

#Codificar columnas categóricas
le = LabelEncoder()

#IP
df[' Source IP'] = df[' Source IP'].apply(lambda ip: int(ipaddress.IPv4Address(ip)))
df[' Destination IP'] = df[' Destination IP'].apply(lambda ip: int(ipaddress.IPv4Address(ip)))

#Fecha
df[' Timestamp'] = pd.to_datetime(df[' Timestamp'])
df.insert(5, ' Year', df[' Timestamp'].dt.year)
df.insert(6, ' Month', df[' Timestamp'].dt.month)
df.insert(7, ' Day', df[' Timestamp'].dt.day)
df.insert(8, ' Hour', df[' Timestamp'].dt.hour)
df.insert(9, ' Minute', df[' Timestamp'].dt.minute)
df.insert(10, ' Second', df[' Timestamp'].dt.second)
df = df.drop(' Timestamp', axis=1)

3. Transformación de tipos de datos: Algunas columnas pueden tener tipos de datos que ocupan más memoria de la necesaria. Por eso, podemos convertir columnas de tipo float64 a float32 para ahorrar memoria. 

In [3]:
# Convertir los tipos de datos de las columnas
for col in df.columns:
    if df[col].dtype == 'float64':
        df[col] = df[col].astype('float32')

4. Guardar el dataset preprocesado: Finalmente, guardaremos el dataset preprocesado en disco en formato Parquet, que es un formato de archivo columnar que proporciona almacenamiento eficiente.

In [4]:
# Guardar el dataset en formato Parquet
df.to_parquet('SynComprimido.parquet')

5. Preprocesamiento de todos los conjuntos de datos disponibles: Aquí, hemos repetido los pasos de las fases anteriores, pero esta vez con todos los conjuntos de datos disponibles a nuestra disposición.

In [3]:
import pandas as pd
import os
import ipaddress
from sklearn.preprocessing import LabelEncoder

# Lista de todos los directorios que contienen archivos CSV
data_dirs = ['DataSets//CICDDoS2019//01-12//', 'DataSets//CICDDoS2019//03-11//']

# Inicializar el codificador de etiquetas
le = LabelEncoder()

for data_dir in data_dirs:
    files = [f for f in os.listdir(data_dir) if f.endswith('.csv')]

    for file in files:
        # Cargar el dataset, eliminar columnas irrelevantes, valores nulos y elementos repetidos
        df = pd.read_csv(data_dir + file, low_memory=False)
        df = df.drop(['Unnamed: 0', 'Flow ID'], axis=1)
        df = df.dropna()
        df = df.drop_duplicates()

        # Transformar las columnas de IP
        df[' Source IP'] = df[' Source IP'].apply(lambda ip: int(ipaddress.IPv4Address(ip)))
        df[' Destination IP'] = df[' Destination IP'].apply(lambda ip: int(ipaddress.IPv4Address(ip)))

        # Transformar la columna de fecha
        df[' Timestamp'] = pd.to_datetime(df[' Timestamp'])
        df.insert(5, ' Year', df[' Timestamp'].dt.year)
        df.insert(6, ' Month', df[' Timestamp'].dt.month)
        df.insert(7, ' Day', df[' Timestamp'].dt.day)
        df.insert(8, ' Hour', df[' Timestamp'].dt.hour)
        df.insert(9, ' Minute', df[' Timestamp'].dt.minute)
        df.insert(10, ' Second', df[' Timestamp'].dt.second)
        df = df.drop(' Timestamp', axis=1)

        # Convertir los tipos de datos de las columnas float64 a float32
        for col in df.columns:
            if df[col].dtype == 'float64':
                df[col] = df[col].astype('float32')

        # Guardar el DataFrame en un archivo Parquet con el mismo nombre que el archivo CSV original
        df.to_parquet('DataSets//CICDDoS2019//Parquet//' + file.replace('.csv', '.parquet'))

KeyboardInterrupt: 